In [1]:
from typing import Dict, List, Set
from sentence_transformers import SentenceTransformer, util

# 載入預訓練的多語言 SBERT 模型
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

c:\Users\op237\miniconda3\envs\boat_and_RecommendationSystem\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 課程數據
courses = {
    # 核心課程
    "A": {"name": "基礎程式設計", "desc": "學習 Python 的基礎語法和簡單程式設計概念", "prereqs": set(), "topics": {"程式設計", "Python"}, "type": "核心"},
    "B": {"name": "資料結構", "desc": "介紹基本的資料結構，如陣列、鏈結串列和樹", "prereqs": {"A"}, "topics": {"程式設計", "資料結構"}, "type": "核心"},
    "C": {"name": "機器學習入門", "desc": "學習機器學習的基本概念和簡單模型，使用 Python 實作", "prereqs": {"A"}, "topics": {"機器學習", "Python", "AI"}, "type": "核心"},
    "D": {"name": "進階程式設計", "desc": "深入探討物件導向程式設計和演算法", "prereqs": {"A"}, "topics": {"程式設計", "演算法"}, "type": "核心"},
    "E": {"name": "人工智慧概論", "desc": "介紹 AI 的基礎理論和應用，不含實作", "prereqs": set(), "topics": {"AI", "理論"}, "type": "核心"},

    # 通識課程
    "GE1": {"name": "科技與社會", "desc": "探討科技對社會的影響，涵蓋 AI 和數位化趨勢", "prereqs": set(), "topics": {"科技", "社會", "AI"}, "type": "通識"},
    "GE2": {"name": "環境科學入門", "desc": "介紹環境問題與可持續發展的基本概念", "prereqs": set(), "topics": {"環境", "科學"}, "type": "通識"},
    "GE3": {"name": "世界經濟", "desc": "介紹國際世界觀，經濟脈絡", "prereqs": set(), "topics": {"國際", "經濟"}, "type": "通識"},
    "GE4": {"name": "數據素養與決策", "desc": "培養數據思維，理解資料背後的意義與決策依據", "prereqs": set(), "topics": {"資料分析", "決策", "數據素養"}, "type": "通識"},
    "GE5": {"name": "心理學概論", "desc": "認識人類行為與思維的基本心理學原理", "prereqs": set(), "topics": {"心理學", "人文"}, "type": "通識"},

    # 選修課程
    "EL1": {"name": "網頁開發基礎", "desc": "學習 HTML、CSS 和 JavaScript 打造簡單網頁", "prereqs": {"A"}, "topics": {"程式設計", "網頁開發"}, "type": "選修"},
    "EL2": {"name": "深度學習概論", "desc": "介紹深度學習的基本原理和應用，使用 Python", "prereqs": {"C"}, "topics": {"機器學習", "AI", "Python"}, "type": "選修"},
    "EL3": {"name": "遊戲設計入門", "desc": "學習遊戲設計基礎，包含程式設計與創意發想", "prereqs": {"A"}, "topics": {"程式設計", "遊戲設計"}, "type": "選修"},

    # ✅ 統計相關課程
    "ST1": {"name": "統計學基礎", "desc": "學習描述統計、機率分布與假設檢定的基本概念", "prereqs": set(), "topics": {"統計", "資料分析", "數學"}, "type": "核心"},
    "ST2": {"name": "機率論", "desc": "探討隨機變數、期望值、機率分布及其應用", "prereqs": {"ST1"}, "topics": {"機率", "統計", "數學"}, "type": "選修"},
    "ST3": {"name": "R 語言資料分析", "desc": "學習使用 R 語言進行資料清理與視覺化", "prereqs": {"ST1"}, "topics": {"統計", "資料分析", "R"}, "type": "選修"},

    # ✅ 管理學系課程
    "MG1": {"name": "管理學原理", "desc": "介紹組織管理的基本原則、領導、規劃與控制", "prereqs": set(), "topics": {"管理", "組織"}, "type": "核心"},
    "MG2": {"name": "行銷學概論", "desc": "探討行銷策略、市場分析與顧客行為", "prereqs": set(), "topics": {"行銷", "管理", "市場"}, "type": "選修"},
    "MG3": {"name": "財務管理", "desc": "學習資金運用、財務報表分析與投資決策", "prereqs": {"MG1"}, "topics": {"財務", "管理", "投資"}, "type": "選修"}



}


# 檢查先修條件是否滿足
def check_prereqs(course_id: str, completed: Set[str]) -> bool:
    return courses[course_id]["prereqs"].issubset(completed)
# 預先將課程描述轉換為語意向量，包含名稱 + 描述 + 主題
course_embeddings: Dict[str, any] = {}
for cid, info in courses.items():
    # 將主題 topics 集合轉為字串
    topic_text = "、".join(info["topics"])
    full_text = f"{info['name']}：{info['desc']}。主題包括：{topic_text}"
    course_embeddings[cid] = sbert_model.encode(full_text, convert_to_tensor=True)


In [8]:
# 推薦課程
def recommend_courses(completed: Set[str], interests: str) -> List[str]:
    # 使用者興趣文字向量化
    user_text = f"{interests}"
    interest_embedding = sbert_model.encode(user_text, convert_to_tensor=True)

    # 計算與每門課的語意相似度
    scored_courses = []
    for cid, info in courses.items():
        if check_prereqs(cid, completed):
            similarity = util.pytorch_cos_sim(interest_embedding, course_embeddings[cid]).item()
            scored_courses.append((cid, similarity, info["type"]))

    # 根據語意相似度與課程類型排序
    scored_courses.sort(key=lambda x: (x[1], {"核心": 0, "選修": 1, "通識": 2}[x[2]]), reverse=True)

    # 推薦結果（最多前 5）
    recommendations = []
    for cid, sim, ctype in scored_courses[:]:
        course = courses[cid]
        rec = f"推薦「{course['name']}」（{ctype}）因為它與你的興趣「{interests}」語意相似度為 {sim:.2f}，且你已滿足先修條件。"
        recommendations.append(rec)

    return recommendations

In [13]:
# 主程式
def main():
    # 模擬用戶資料
    completed_courses = {"MG1"}  # 已修過基礎程式設計
    user_interests = "財務管理"  # 用自然語言描述興趣

    # 推薦課程
    recommendations = recommend_courses(completed_courses, user_interests)

    # 顯示結果
    print("根據你的背景與興趣，推薦課程如下：")
    if recommendations:
        for r in recommendations[:5]:
            print(r)
    else:
        print("目前沒有符合條件的推薦課程。")

if __name__ == "__main__":
    main()

根據你的背景與興趣，推薦課程如下：
推薦「財務管理」（選修）因為它與你的興趣「財務管理」語意相似度為 0.76，且你已滿足先修條件。
推薦「管理學原理」（核心）因為它與你的興趣「財務管理」語意相似度為 0.45，且你已滿足先修條件。
推薦「行銷學概論」（選修）因為它與你的興趣「財務管理」語意相似度為 0.36，且你已滿足先修條件。
推薦「數據素養與決策」（通識）因為它與你的興趣「財務管理」語意相似度為 0.29，且你已滿足先修條件。
推薦「世界經濟」（通識）因為它與你的興趣「財務管理」語意相似度為 0.27，且你已滿足先修條件。
